### CUSTOM ACTIVATION FUNCTIONS, INITIALIZERS, REGULARIZERS, AND CONSTRAINTS

+ Most keras functionalities such as below can be customized.
     + losses
     + regularizers
     + constraints
     + initializers
     + metrics
     + activation functions
     + layers
     + even full model
+ Here are custom functions that are equivalent to
    + activation function - `keras.activations.softplus() or tf.nn.softplus()`
    + Glorot Initializer - `keras.initializers.glorot_normal()`
    + l1 regularizers - `keras.regularizers.l1(0.01)`
    + custom constraint that ensures weights are all positives - `keras.constraints.nonneg() or tf.nn.relu()`

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
## fetch the data.
housing = fetch_california_housing()
# housing.data
# housing.target
print(housing.keys())
print()
print("====================================Complete Dataset======================================")
## X_train_full, X_test, y_train_full, y_test
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1,1), random_state = 42)
print(X_train_full.shape)
print(X_test.shape)
print(y_train_full.shape)
print(y_test.shape)
print()
print("====================================Dataset After Splitting to Validation Set======================================")
## X_valid, X_test, y_valid, y_test
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print()

print("=================================Standardize the Data==============================================")
## Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled.shape)
X_valid_scaled = scaler.transform(X_valid)
print(X_valid_scaled.shape)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.shape)

## defining the input shape
input_shape = X_train.shape[1:]
print(input_shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

====================================Complete Dataset======================================
(15480, 8)
(5160, 8)
(15480, 1)
(5160, 1)

====================================Dataset After Splitting to Validation Set======================================
(11610, 8)
(3870, 8)
(11610, 1)
(3870, 1)

=================================Standardize the Data==============================================
(11610, 8)
(3870, 8)
(5160, 8)
(8,)


In [10]:
## activation function
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)

## glorot initialization
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

## l1 regularization
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

## positive metrics for weights.
def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [11]:
## defining the final layer of the model
## all hyperpaarameter are custom made.
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

## defining the model
print("=======================DEFINING THE MODEL=========================")
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=my_l1_regularizer,
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])
print("=======================COMPILE THE MODEL===========================")
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

print("=======================TRAIN THE MODEL=============================")
model.fit(X_train_scaled, y_train, epochs=20,
          validation_data=(X_valid_scaled, y_valid))

=======================DEFINING THE MODEL=========================
=======================COMPILE THE MODEL===========================
=======================TRAIN THE MODEL=============================
Epoch 1/20
363/363 [==============================] - 4s 3ms/step - loss: 1.5194 - mae: 0.8601 - val_loss: inf - val_mae: inf
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6237 - mae: 0.5165 - val_loss: inf - val_mae: inf
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5559 - mae: 0.4999 - val_loss: inf - val_mae: inf
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5270 - mae: 0.4926 - val_loss: inf - val_mae: inf
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5130 - mae: 0.4893 - val_loss: inf - val_mae: inf
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5031 - mae: 0.4872 - val_loss: 2.4822 - val_mae: 0.5001
Epoch 7/20
363/363 [===============

In [12]:
## save the model
model.save("my_model_with_many_custom_parts.h5")

Now when you load the model, you have to specify the hyperparameters that were defined and used before

In [13]:
model_loaded = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "my_l1_regularizer": my_l1_regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

In [14]:
model_loaded.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.4642 - mae: 0.4737 - val_loss: 0.5867 - val_mae: 0.4624
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.4613 - mae: 0.4709 - val_loss: 0.6076 - val_mae: 0.4571


In [16]:
## defineing the l1 regularization class.
## this is to save the scores when saving the model
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}
    
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=MyL1Regularizer(0.01),
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.6164 - mae: 0.8978 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.6711 - mae: 0.5363 - val_loss: inf - val_mae: inf


In [17]:
## saving the model here again
model.save("my_model_with_many_custom_parts_with_l1_regularizer.h5")

In [18]:
## when you are loading the model again
## we need to specify the hyperparameters that we were defined and used before here
model = keras.models.load_model(
    "my_model_with_many_custom_parts_with_l1_regularizer.h5",
    custom_objects={
       "MyL1Regularizer": MyL1Regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.5852 - mae: 0.5089 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.5469 - mae: 0.4962 - val_loss: inf - val_mae: inf


In [24]:
model.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.040859472>]